In [1]:
#loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# setting plt to inline
%matplotlib inline

In [2]:
# importing sqlite3
import sqlite3

# conneting to db and creating cursor
conn = sqlite3.Connection('data/movies.db')
cur = conn.cursor()

# creating query functions
def fetcha(q):
    return cur.execute(q).fetchall()
def fetcho(q):
    return cur.execute(q).fetchone()

In [3]:
# getting table names
q = """SELECT name FROM sqlite_master 
WHERE type IN ('table','view') 
AND name NOT LIKE 'sqlite_%'
ORDER BY 1"""
fetcha(q)

[('bom_movie_gross',),
 ('imdb_name_basic',),
 ('imdb_name_basics',),
 ('imdb_title_akas',),
 ('imdb_title_basics',),
 ('imdb_title_crew',),
 ('imdb_title_principals',),
 ('imdb_title_ratings',),
 ('rotten_tomatoes_critic_reviews',),
 ('rotten_tomatoes_movies',),
 ('tmdb_movies',),
 ('tn_movie_budgets',)]

In [5]:
q = "PRAGMA table_info('rotten_tomatoes_movies')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 1),
 (1, 'rotten_tomatoes_link', 'TEXT', 0, None, 0),
 (2, 'movie_title', 'TEXT', 0, None, 0),
 (3, 'movie_info', 'TEXT', 0, None, 0),
 (4, 'critics_consensus', 'TEXT', 0, None, 0),
 (5, 'content_rating', 'TEXT', 0, None, 0),
 (6, 'genres', 'TEXT', 0, None, 0),
 (7, 'directors', 'TEXT', 0, None, 0),
 (8, 'authors', 'TEXT', 0, None, 0),
 (9, 'actors', 'TEXT', 0, None, 0),
 (10, 'original_release_date', 'TEXT', 0, None, 0),
 (11, 'streaming_release_date', 'TEXT', 0, None, 0),
 (12, 'runtime', 'REAL', 0, None, 0),
 (13, 'production_company', 'TEXT', 0, None, 0),
 (14, 'tomatometer_status', 'TEXT', 0, None, 0),
 (15, 'tomatometer_rating', 'REAL', 0, None, 0),
 (16, 'tomatometer_count', 'REAL', 0, None, 0),
 (17, 'audience_status', 'TEXT', 0, None, 0),
 (18, 'audience_rating', 'REAL', 0, None, 0),
 (19, 'audience_count', 'REAL', 0, None, 0),
 (20, 'tomatometer_top_critics_count', 'INTEGER', 0, None, 0),
 (21, 'tomatometer_fresh_critics_count', 'INTEGER', 0, N

In [6]:
q = "PRAGMA table_info('tn_movie_budgets')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 1),
 (1, 'id', 'INTEGER', 0, None, 0),
 (2, 'release_date', 'TEXT', 0, None, 0),
 (3, 'movie', 'TEXT', 0, None, 0),
 (4, 'production_budget', 'TEXT', 0, None, 0),
 (5, 'domestic_gross', 'TEXT', 0, None, 0),
 (6, 'worldwide_gross', 'TEXT', 0, None, 0)]

In [10]:
# creating initial dataframe by joining the four tables and printing first row
q = """SELECT*FROM rotten_tomatoes_movies AS rt
       JOIN tn_movie_budgets AS tn
       ON rt.movie_title = tn.movie
       
       """
df = pd.DataFrame(fetcha(q))
df.columns = [i[0] for i in cur.description]
df.head(1)     

,idx,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,...,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,idx,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,43,73,76,416,17,"Feb 12, 2010",Percy Jackson & the Olympians: The Lightning T...,"$95,000,000","$88,768,303","$223,050,874"


In [11]:
df.loc[df['movie_title'] == 'Mad Max']

,idx,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,...,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,idx,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
2785,9910,m/mad_max,Mad Max,"In a not-too-distant dystopian future, when ma...",Staging the improbable car stunts and crashes ...,R,"Action & Adventure, Cult Movies, Science Ficti...",George Miller,"James McCausland, George Miller","Mel Gibson, Joanne Samuel, Tim Burns, Hugh Kea...",...,7,56,6,5613,14,"Mar 21, 1980",Mad Max,"$200,000","$8,750,000","$99,750,000"


In [48]:
split_list = []
for genre in df.genres:
    split_list += genre.split(',')

raw_genres = list(set(split_list))
raw_genres

['Science Fiction & Fantasy',
 ' Horror',
 ' Faith & Spirituality',
 ' Art House & International',
 ' Comedy',
 ' Science Fiction & Fantasy',
 'Kids & Family',
 'Animation',
 ' Sports & Fitness',
 ' Special Interest',
 ' Musical & Performing Arts',
 'Romance',
 ' Animation',
 'Drama',
 ' Western',
 ' Cult Movies',
 'Cult Movies',
 'Art House & International',
 ' Mystery & Suspense',
 ' Romance',
 ' Anime & Manga',
 'Documentary',
 'Classics',
 'Western',
 ' Classics',
 'Comedy',
 ' Documentary',
 'Mystery & Suspense',
 'Musical & Performing Arts',
 ' Kids & Family',
 ' Television',
 ' Drama',
 'Horror',
 ' Gay & Lesbian',
 'Action & Adventure']

In [51]:
split_list = []
for genre in df.genres:
    split_list += [g.strip().lower() for g in genre.split(',')]
    
genres = list(set(split_list))
genres

['animation',
 'special interest',
 'mystery & suspense',
 'classics',
 'western',
 'horror',
 'art house & international',
 'gay & lesbian',
 'documentary',
 'anime & manga',
 'cult movies',
 'faith & spirituality',
 'romance',
 'sports & fitness',
 'kids & family',
 'science fiction & fantasy',
 'comedy',
 'television',
 'musical & performing arts',
 'drama',
 'action & adventure']

In [52]:
len(genres)

21

In [53]:
df[(df['genres'] == 'Science Fiction & Fantasy') |\
   (df['genres'] == ' Science Fiction & Fantasy')]

,idx,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,...,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,idx,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
151,418,m/1006364-dune,Dune,"In the year 10191, a spice called melange is t...",This truncated adaptation of Frank Herbert's s...,PG-13,Science Fiction & Fantasy,David Lynch,None,"Francesca Annis, José Ferrer, Leonardo Cimino,...",...,9,26,23,1297,98,"Dec 14, 1984",Dune,"$45,000,000","$27,447,471","$27,447,471"
471,1239,m/1089671-13th_floor,The Thirteenth Floor,A man awakens to discover a bloody shirt in hi...,Bad script and confusing plot undermine the mo...,R,Science Fiction & Fantasy,Josef Rusnak,"Josef Rusnak, Ravel Centeno-Rodriguez","Craig Bierko, Armin Mueller-Stahl, Gretchen Mo...",...,19,19,45,2960,61,"May 28, 1999",The Thirteenth Floor,"$16,000,000","$11,810,854","$11,810,854"
572,1379,m/1112951-time_machine,The Time Machine,Scientist and inventor Alexander Hartdegen (Gu...,This Machine has all the razzle-dazzles of mod...,PG-13,Science Fiction & Fantasy,"Simon Wells, Gore Verbinski","John Logan, Simon Wells","Guy Pearce, Samantha Mumba, Jeremy Irons, Orla...",...,34,44,109,582,83,"Mar 8, 2002",The Time Machine,"$80,000,000","$56,684,819","$98,983,590"
1205,3759,m/blood_of_heroes,The Blood of Heroes,Another woman (Joan Chen) joins a future gladi...,None,R,Science Fiction & Fantasy,None,None,"Rutger Hauer, Joan Chen, Vincent D'Onofrio, De...",...,5,1,7,3950,51,"Feb 23, 1990",The Blood of Heroes,"$7,700,000","$882,290","$882,290"
1582,5221,m/day_the_earth_stood_still,The Day the Earth Stood Still,"Klaatu (Keanu Reeves), an extraterrestrial vis...","Heavy on special effects, but without a cohere...",PG-13,Science Fiction & Fantasy,Scott Derrickson,"Scott Derrickson, David Scarpa, Edmund H. North","Keanu Reeves, Jennifer Connelly, Jaden Smith, ...",...,49,40,155,5164,65,"Sep 18, 1951",The Day the Earth Stood Still,"$1,200,000","$3,700,000","$3,700,000"
1583,5221,m/day_the_earth_stood_still,The Day the Earth Stood Still,"Klaatu (Keanu Reeves), an extraterrestrial vis...","Heavy on special effects, but without a cohere...",PG-13,Science Fiction & Fantasy,Scott Derrickson,"Scott Derrickson, David Scarpa, Edmund H. North","Keanu Reeves, Jennifer Connelly, Jaden Smith, ...",...,49,40,155,571,72,"Dec 12, 2008",The Day the Earth Stood Still,"$80,000,000","$79,366,978","$237,370,350"
2807,9988,m/maleficent_2014,Maleficent,"As a beautiful young woman of pure heart, Male...",Angelina Jolie's magnetic performance outshine...,PG,Science Fiction & Fantasy,Robert Stromberg,Linda Woolverton,"Angelina Jolie, Sharlto Copley, Elle Fanning, ...",...,55,145,125,83,84,"May 30, 2014",Maleficent,"$180,000,000","$241,407,328","$758,536,735"
2935,10443,m/mirror_mirror_2012,Mirror Mirror,"Snow White (Lily Collins), an orphaned princes...","Like most of Tarsem Singh's films, Mirror Mirr...",PG,Science Fiction & Fantasy,Tarsem Singh,"Melisa Wallack, Jason Keller, Marc Klein","Julia Roberts, Lily Collins, Armie Hammer, Sea...",...,40,99,98,506,7,"Mar 30, 2012",Mirror Mirror,"$85,000,000","$64,935,167","$173,613,482"
3752,13492,m/snowpiercer,Snowpiercer,A post-apocalyptic ice age forces humanity's l...,Snowpiercer offers an audaciously ambitious ac...,R,Science Fiction & Fantasy,Bong Joon Ho,"Bong Joon Ho, Kelly Masterson","Chris Evans, Song Kang Ho, Jamie Bell, John Hu...",...,47,237,16,1526,27,"Jun 27, 2014",Snowpiercer,"$40,000,000","$4,563,029","$88,069,880"
3792,13670,m/species,Species,When government scientist Xavier Fitch (Ben Ki...,Species shows flashes of the potential to blen...,R,Science Fiction & Fantasy,Roger Donaldson,"Dennis Feldman, Roger Donaldson","Ben Kingsley, Michael Madsen, Alfred Molina, F...",...,17,28,37,1650,51,"Jul 7, 1995",Species,"$35,000,000","$60,054,449","$113,354,449"
